<a href="https://colab.research.google.com/github/dijahanga/DL_Approach_To_Process_Mining/blob/main/PGraphDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import math
import numpy as np
import pydotplus as pydot
from graphviz import Digraph
import copy
import csv
from google.colab import files

In [ ]:
class helper:
  def __init__(self):
    self.i= 0
  def datasetListMergeMinus(self, dataset, subset):
    wholeData = 1
    for m in dataset:
      if not m.equals(subset):
        if type(wholeData) is int:
          wholeData = m
        else:
          wholeData = wholeData.append(m)
    return wholeData
  def multiDimStrToUpper(self, string):
    nstring = []
    for strns in string:
      nstring.append([x.upper() for x in strns])
    return nstring
  def multiDimStrToLower(self, string):
    nstring = []
    for strns in string:
      nstring.append([x.lower() for x in strns])
    return nstring
  def oneDimStrToLower(self, string):
    nstring = []
    for i in range(0, len(string)):
      nstring.append(string[i].lower())
    return nstring
  def grabEventsFromHeader(self, header):
    evs = []
    c = 0
    for ev in header:
      if c > 0:
        try:
          num = int(ev)
        except:
          evs.append(ev)
      c = c + 1
    return evs

  def rowIsFirst(self, row, activities, headers):
    foundValues = [];
    for i in range(0,len(headers)):
      val = headers[i]
      if row[val] in activities:
        foundValues.append(row[val])
    if (len(foundValues) == 0):
      return True
    return False

  def rowIsLast(self, row, evName):
    rowEv = row.idxmax()
    try:
      if rowEv.lower() == evName.lower():
        return True
    except:
      return False
    return False

  def divideMatrix(self, matrix):
    headers = list(matrix.columns.values)
    leftHeaders = []
    rightHeaders = []
    for i in range(0,len(headers)):
      ev = headers[i]
      try:
          num = int(ev)        
          leftHeaders.append(ev)        
      except:
          rightHeaders.append(ev)
    leftData = matrix[leftHeaders]
    rightData = matrix[rightHeaders]
    return [leftData, rightData]

In [ ]:
class resultGraphing:
  def __init__(self):
    self.i= 0
    
  def decomposeResult(self, results):
    helper_ = helper()
    headers = list(results.columns.values)
    events = helper_.grabEventsFromHeader(headers)
    matrices = []
    matricesLeft = []
    lastIndex = 0
    totalBegin = 0
    ind = 0
    newMatrix = helper_.divideMatrix(results)
    for index, row in newMatrix[1].iterrows():
      ind = index
      rowIsLast = helper_.rowIsLast(row, "end")
      if rowIsLast:
        sequenceList = newMatrix[1].iloc[lastIndex:index+1, :]
        sequenceListLeft = newMatrix[0].iloc[lastIndex:index+1, :]
        matrices.append(sequenceList)
        matricesLeft.append(sequenceListLeft)
        lastIndex = index + 1
    return matrices, matricesLeft

  def rowIsAnewSequence(self, row):
    try:
      return math.isnan(row["1"])
    except:
      return False

  def linkAndProbabilities(self, matrices, count = 0):
    links = []
    probabilities = []
    sequences = []
    uniqueEvs = []
    for i in range(0,len(matrices)):
      thisMatrix = matrices[i]
      lastEvent = "Start"
      sequence = []
      for index, row in thisMatrix.iterrows():
        row = pd.to_numeric(row)
        #print(row) # prints the rows
        evName = row.idxmax(axis=1) # picks event with the highest probability
        link = lastEvent + "<-->" + evName
        sequence.append(evName)
        if link not in links:
          if lastEvent != evName:
            if not (lastEvent.lower() == "start" and evName.lower() == "end"):
              links.append(link)
              prob = row[evName]
              probabilities.append(prob)
        lastEvent = evName
      # The Last element is End and undesirable
      sequence.pop()
      sequences.append(sequence)
    return links, probabilities, sequences    

  def linkAndProbabilities_2(self, seqs, probs):
    links = []
    probabilities = []
    sequences = []
    for i in range(0,len(probs)):
      lastEvent = "Start"
      sequence = []
      considprob = probs[i]
      cols = considprob.columns.values
      for index, row in considprob.iterrows():
        evName = row.idxmax()
        link = lastEvent + "<-->" + evName
        sequence.append(evName)
        prob = considprob[evName].max(axis=0)
        if link not in links:
          links.append(link)            
          probabilities.append(prob)              
        lastEvent = evName
      sequence.pop() #
      sequences.append(sequence)
    return links, probabilities, sequences


  def linkAndProbabilities_3(self, seqs, probs):
    links = []
    probabilities = []
    sequences = []
    uniqueEvs = []
    orphanLog = []
    pointedToLog = []
    allowedActivities = probs[0].columns.values
    for i in range(0,len(seqs)):
      thisMatrix = probs[i]
      lastEvent = "Start"
      sequence = []
      completeSequence = seqs[i].iloc[-1]
      for j in range(1, len(completeSequence)):
        if completeSequence[j] in allowedActivities:
          evName = completeSequence[j]
          sequence.append(evName)
          link = lastEvent + "<-->" + evName
          prob = probs[i][evName].max(axis=0)
          if link not in links:
            links.append(link)
            probabilities.append(prob)
          lastEvent = evName
        else:
          link = lastEvent + "<-->END"
          prob = 100
          if link not in links:
            links.append(link)            
            probabilities.append(prob)              
          lastEvent = "END"
          break
      sequences.append(sequence)
    return links, probabilities, sequences
            
  def drawGraph(self, transitions, probabilities, counter = 0):    
    G = Digraph('process_model', filename='graph_'+str(counter)+'.gv')
    G.attr(rankdir='LR', size='7,5')
    G.attr('node', shape='doublecircle', style="filled", fillcolor="grey")
    G.node('Start')
    G.node('End')
    G.attr('node', shape='box', style="bold")
    for i in range(0,len(transitions)):
      G.attr('edge', style="bold", penwidth='3.0', label=str(probabilities[i]))
      fromto = transitions[i].split("<-->")
      G.edge(fromto[0], fromto[1])
    G.view()
    return G

  def getEventSequence(self, data, X_label, Y_label):
    currentX_label = ''
    sequences = []
    sequence = []
    for index, row in data.iterrows():
      if currentX_label == row[X_label]:
        sequence.append(row[Y_label])
      else:
        if len(sequence) > 1:
          sequences.append(sequence)
        sequence = []
        sequence.append(row[Y_label])
      currentX_label = row[X_label]
    sequences.append(sequence)
    return sequences    
  
  # def getEventSequence(self, data, X_label, Y_label, caseNumber = None):
  #   currentX_label = ''
  #   sequences = []
  #   sequence = []
  #   for index, row in data.iterrows():
  #     push = False
  #     if caseNumber is not None:
  #       if str(currentX_label) == str(caseNumber) or str(row[X_label]) == str(caseNumber):
  #         push = True
  #     else:
  #       push = True;

  #     if currentX_label == row[X_label]:
  #       if push:
  #         sequence.append(row[Y_label])
  #     else:
  #       if len(sequence) > 1:
  #         sequences.append(sequence)        
  #       if push :
  #         sequence = []
  #         sequence.append(row[Y_label])
  #     currentX_label = row[X_label]
  #   sequences.append(sequence)
  #   return sequences

In [ ]:
class processDrift:
  def __init__(self, driftedSequences):    
    self.driftedSequences = driftedSequences
    
  def findBestCandidate(self, sequence):
    self.originalSequence = sequence
    scores = [0] * len(self.driftedSequences)
    positionalDrifts = []
    i = 0
    for aSequence in self.driftedSequences:
      x, y = self.positionalScore(sequence, aSequence)
      scores[i] = x
      drift = y
      positionalDrifts.append(drift)
      i = i + 1
    bestMatch = scores.index(max(scores))
    return self.driftedSequences[bestMatch], positionalDrifts[bestMatch]
    
  def positionalScore(self, sequence, driftedSequence):
    score = 0
    counter = 0
    positionalDrift = []
    for event in sequence:
      try:
        if driftedSequence[counter] == event:
          score = score + 1
        else:
          positionalDrift.append(counter)
      except:
        break
      counter = counter + 1
    if score == 0:
      return 0, positionalDrift
    return (score/len(sequence)), positionalDrift

  def drawGraph(self, transitions, drift, diag_counter = 0):
    G = Digraph('process_model'+str(diag_counter)+'.gv')
    G.attr(rankdir='LR', size='7,5')
    G.attr('node', shape='doublecircle', style="filled", fillcolor="grey")
    G.node('Start')
    G.node('End')
    G.attr('node', shape='box', style="bold")
    lastEv = "Start"
    counter = 0
    usedEvs = []
    for ev in transitions:
      if ev == "END":
        counter = counter + 1
        continue
      if ev in usedEvs:
        counter = counter + 1
        continue
      usedEvs.append(ev)
      if counter in drift:
        G.attr('node', shape='circle', style="dotted")
        G.attr('edge', style="bold", penwidth='1.0')
      else:
        G.attr('node', shape='box', style="bold")
        G.attr('edge', style="bold", penwidth='3.0')
      G.edge(lastEv, ev)
      lastEv = ev
      counter = counter + 1
    G.edge(lastEv, "End")
    G.view()
    return G

In [ ]:
originalDataset = pd.read_csv('referenceLogRP_0.csv', low_memory=False)
#originalDataset = pd.read_csv('bpic_1.csv')
originalDataset.drop(originalDataset.columns[0], axis = 1, inplace = True)

In [ ]:
resultDataset = pd.read_csv('detectionLogRP_1_0.csv', low_memory=False)
#resultDataset = pd.read_csv('bpic_2.csv')
resultDataset.drop(resultDataset.columns[0], axis = 1, inplace = True)

In [ ]:
_resultGraphing = resultGraphing()

In [ ]:
print(originalDataset)

In [ ]:
seq = _resultGraphing.getEventSequence(originalDataset, 'case', 'event')

In [ ]:
driftprobs, driftseqs = _resultGraphing.decomposeResult(resultDataset)
link_2, probabilities_2, sequences_2 = _resultGraphing.linkAndProbabilities_3(driftseqs, driftprobs)
link, probabilities, sequences = _resultGraphing.linkAndProbabilities(driftprobs)

In [ ]:
original = seq[0]

In [ ]:
#drift = sequences
drift_2 = sequences_2

In [ ]:
processDrift_ = processDrift(drift_2)
j, k = processDrift_.findBestCandidate(original)
processDrift_.drawGraph(original, [],)

In [ ]:
processDrift_.drawGraph(j, k, 1)

In [ ]:
processDrift_2 = processDrift(drift_2)
j, k = processDrift_2.findBestCandidate(original)
processDrift_2.drawGraph(original, [],)

In [ ]:
processDrift_2.drawGraph(j, k, 1)